In [ ]:
!pip install transformers
!pip install -U sentence-transformers
!pip install gensim

     |████████████████████████████████| 1.3MB 12.6MB/s 
     |████████████████████████████████| 2.9MB 51.2MB/s 
     |████████████████████████████████| 890kB 48.5MB/s 
     |████████████████████████████████| 1.1MB 52.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=702f0c63552ac46a859f4304d6c60ae5442d9c3c7795bed97c1520a142199c8e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 71kB 6.8MB/s 
     |████████████████████████████████| 1.1MB 19.4MB/s 
     |████████████████████████████████| 3.0MB 43.4MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.8-cp36-none-any.whl size=101996 sha256=63fc0bb364cff9fcd5c506810b3f091aefbd573e4ea6447d17cabcb7180015eb
  Stored in directory: /root/.cache/pip/wheels/27/ec/b3/d12cc8e4daf77846db6543033d3a5642f204c0320b15945647
Successfully built sent

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/'My Drive'/'CS229'
# %rm -rf fid_test_pix2pix_3
%ls

/content/drive/.shortcut-targets-by-id/1sPYcNDVL_owY8VCbxPtp6HF2oVELOYJo/CS229
 atis/                                     generate_bert_encodings_1.py
 atis_logs/                                generate_bert_encodings.py
 atis_logs_freeze/                         generate_word2vec_encodings.py
 atis_results/                             load_data_1.py
'Copy of Fine-tune BERT (ATIS).ipynb'      load_data.py
'Copy of Fine-tune BERT (SNIPS).ipynb'     LSTM_Model/
'CS 229 Project Ideas.gdoc'               'Project Planning.gdoc'
 data/                                     __pycache__/
 data.gsheet                               Resources.gdoc
 embeddings/                               snips/
'Fine-tune BERT (ATIS) (Freeze).ipynb'     snips_distil_logs/
'Fine-tune BERT (ATIS).ipynb'              snips_electra_logs/
'Fine-tune BERT.ipynb'                     snips_logs/
'Fine-tune BERT (SNIPS) - Copy.ipynb'      snips_logs_copy/
'Fine-tune BERT (SNIPS) (Freeeze).ipynb'   snips_logs_freeeze/
'Fi

In [ ]:
import os, sys
import argparse
import numpy as np
import torch
import transformers
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, f1_score
from load_data import *

In [ ]:
num_labels = 8
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels, return_dict=True)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
DATASET = 'atis'
TRAIN_PATH = f'./{DATASET}/{DATASET}_train_actual.csv'
TEST_PATH = f'./{DATASET}/{DATASET}_test_actual.csv'

In [ ]:
data_loader = DataLoader(TRAIN_PATH, None, TEST_PATH)
data_loader.split_train_valid(valid_size=0.05, keep_class_ratios=True)
X_train, y_train = data_loader.get_train_data()
X_valid, y_valid = data_loader.get_valid_data()
X_test, y_test = data_loader.get_test_data()

In [ ]:
X_train = tokenizer(X_train, truncation=True, padding=True)
X_valid = tokenizer(X_valid, truncation=True, padding=True)
X_test = tokenizer(X_test, truncation=True, padding=True)

In [ ]:
class IntentDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        # sample_X = self.X[idx]
        # sample_y = self.y[idx]
        # sample = {
        #     'text': sample_X,
        #     'label': sample_y,
        # }
        # return sample
        item = {key: torch.tensor(val[idx]) for key, val in self.X.items()}
        item['labels'] = torch.tensor(self.y[idx])
        return item

In [ ]:
# def tokenize(batch):
#     return tokenizer(batch['text'], padding=True, truncation=True)

# train_dataset = IntentDataset(X_train, y_train)
# train_dataset = train_dataset.map(tokenize, batched=True, batch_size=batch_size)
# # test_dataset = test_dataset.map(tokenize, batched=True, batch_size=batch_size)
# train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
# # test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
train_dataset = IntentDataset(X_train, y_train)
valid_dataset = IntentDataset(X_valid, y_valid)
test_dataset = IntentDataset(X_test, y_test)

In [ ]:
# def compute_metrics(pred):
#     labels = pred.label_ids
#     preds = pred.predictions.argmax(-1)
#     precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
#     acc = accuracy_score(labels, preds)
#     return {
#         'accuracy': acc,
#         'f1': f1,
#         'precision': precision,
#         'recall': recall
#     }

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
    }

In [ ]:
batch_size = 16
lr = 1e-5
num_epochs = 20
weight_decay = 0.0001

In [ ]:
model.train()

# or param in model.base_model.parameters():
#     param.requires_grad = False

training_args = TrainingArguments(
    output_dir=f'./{DATASET}_results',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=1,
    learning_rate=lr,
    warmup_steps=500,
    weight_decay=weight_decay,
    evaluate_during_training=True,
    logging_dir=f'./{DATASET}_logs/{num_epochs}_{batch_size}_{lr}_{weight_decay}',
    load_best_model_at_end=True,
    metric_for_best_model='eval_accuracy',
    save_steps=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset
)

/usr/local/lib/python3.6/dist-packages/transformers/training_args.py:332: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,


In [ ]:
trainer.train()

{'loss': 0.9082901000976562, 'learning_rate': 1e-05, 'epoch': 1.7421602787456445, 'total_flos': 273283026432000, 'step': 500}



{'eval_loss': 0.12189359030847953, 'eval_accuracy': 0.9710743801652892, 'eval_f1': 0.9679165574734537, 'epoch': 1.7421602787456445, 'total_flos': 273283026432000, 'step': 500}



{'loss': 0.051310302734375, 'learning_rate': 9.045801526717559e-06, 'epoch': 3.484320557491289, 'total_flos': 546566052864000, 'step': 1000}



{'eval_loss': 0.030535500316711314, 'eval_accuracy': 0.9917355371900827, 'eval_f1': 0.9917355371900827, 'epoch': 3.484320557491289, 'total_flos': 546566052864000, 'step': 1000}



{'loss': 0.01217724609375, 'learning_rate': 8.091603053435115e-06, 'epoch': 5.2264808362369335, 'total_flos': 819849079296000, 'step': 1500}


...

{'loss': 0.00174102783203125, 'learning_rate': 1.4122137404580156e-06, 'epoch': 17.421602787456447, 'total_flos': 2732830264320000, 'step': 5000}



{'eval_loss': 0.04465587923174805, 'eval_accuracy': 0.9917355371900827, 'eval_f1': 0.9917355371900827, 'epoch': 17.421602787456447, 'total_flos': 2732830264320000, 'step': 5000}



{'loss': 0.00044921875, 'learning_rate': 4.5801526717557257e-07, 'epoch': 19.16376306620209, 'total_flos': 3006113290752000, 'step': 5500}



{'eval_loss': 0.04789126552509703, 'eval_accuracy': 0.9917355371900827, 'eval_f1': 0.9917355371900827, 'epoch': 19.16376306620209, 'total_flos': 3006113290752000, 'step': 5500}




TrainOutput(global_step=5740, training_loss=0.08642506350208243)

In [ ]:
trainer.evaluate(test_dataset)


{'eval_loss': 0.04259951107273082, 'eval_accuracy': 0.99375, 'eval_f1': 0.9951257533316357, 'epoch': 20.0, 'total_flos': 3137289143439360, 'step': 5740}


{'epoch': 20.0,
 'eval_accuracy': 0.99375,
 'eval_f1': 0.9951257533316357,
 'eval_loss': 0.04259951107273082,
 'total_flos': 3137289143439360}